# App Metrics

In [68]:
# all 100, all test set 1

import pandas as pd
import os

metrics_dir = os.path.join("..", "metrics")

# Load and label
rf_df = pd.read_csv(os.path.join(metrics_dir, "rf_pca_augmented_metrics.csv"))
rf_df["Model_Label"] = "RF (PCA)"

xgb_df = pd.read_csv(os.path.join(metrics_dir, "xgboost_augmented_metrics.csv"))
xgb_df = xgb_df[xgb_df["test_set"] == 1].copy()
xgb_df["Model_Label"] = "XGBoost (PCA)"
xgb_df = xgb_df[xgb_df["noise_level"] != 50]

cnn_df = pd.read_csv(os.path.join(metrics_dir, "cnn_original_augmented_metrics.csv"))
cnn_df = cnn_df[cnn_df["test_set"] == 1].copy()
cnn_df["Model_Label"] = "CNN"

resnet_df = pd.read_csv(os.path.join(metrics_dir, "resnet50_augmented_metrics.csv"))
resnet_df = resnet_df[resnet_df["test_set"] == 1].copy()
resnet_df["Model_Label"] = "ResNet"

# Rename 'confusion' → 'confusion_matrix' if exists
for df in [xgb_df, cnn_df, resnet_df]:
    if "confusion" in df.columns:
        df.rename(columns={"confusion": "confusion_matrix"}, inplace=True)
    elif "confusion_matrix" not in df.columns:
        df["confusion_matrix"] = "[]"

# Rename precision_overall and recall_overall for CNN, XGBoost, resnet
resnet_df.rename(columns={"precision_overall": "precision", "recall_overall": "recall"}, inplace=True)
cnn_df.rename(columns={"precision_overall": "precision", "recall_overall": "recall"}, inplace=True)
xgb_df.rename(columns={"precision_overall": "precision", "recall_overall": "recall"}, inplace=True)

# Combine and save
combined_df = pd.concat([rf_df, xgb_df, cnn_df, resnet_df], ignore_index=True)
combined_df.fillna(0.0, inplace=True)
combined_df.to_csv("../metrics/combined_app_metrics.csv", index=False)

print(combined_df.columns.tolist())

['blur_size', 'noise_level', 'accuracy', 'f1', 'precision', 'recall', 'f1_immune', 'precision_immune', 'recall_immune', 'f1_other', 'precision_other', 'recall_other', 'f1_stromal', 'precision_stromal', 'recall_stromal', 'f1_tumour', 'precision_tumour', 'recall_tumour', 'confidence_overall', 'confidence_immune_avg', 'confidence_immune_std', 'confidence_other_avg', 'confidence_other_std', 'confidence_stromal_avg', 'confidence_stromal_std', 'confidence_tumour_avg', 'confidence_tumour_std', 'count_pred_immune', 'count_pred_other', 'count_pred_stromal', 'count_pred_tumour', 'confusion_matrix', 'Model_Label', 'test_set']


# Report Metrics (Full Data)

In [9]:
import pandas as pd
import os

metrics_dir = os.path.join("..", "metrics")

# === Load and label RF variants ===
rf_all = pd.read_csv(os.path.join(metrics_dir, "rf_augmented_metrics.csv"))
rf_pixel = rf_all[(rf_all["HOG"] == False) & (rf_all["PCA"] == False)].copy()
rf_pixel["Model_Label"] = "RF (pixels)"

rf_pca = rf_all[(rf_all["PCA"] == True)].copy()
rf_pca["Model_Label"] = "RF (PCA)"

rf_hog = rf_all[(rf_all["HOG"] == True)].copy()
rf_hog["Model_Label"] = "RF (HOG)"

# === Load and label remaining models ===
xgb_df = pd.read_csv(os.path.join(metrics_dir, "xgboost_augmented_metrics.csv"))
xgb_df["Model_Label"] = "XGBoost (PCA)"
xgb_df = xgb_df[xgb_df["noise_level"] != 50]

cnn_df = pd.read_csv(os.path.join(metrics_dir, "cnn_original_augmented_metrics.csv"))
cnn_df["Model_Label"] = "CNN"

resnet_df = pd.read_csv(os.path.join(metrics_dir, "resnet50_augmented_metrics.csv"))
resnet_df["Model_Label"] = "ResNet"

# === Keep all test sets for report ===

# === Rename confusion column if needed ===
for df in [xgb_df, cnn_df, resnet_df]:
    if "confusion" in df.columns:
        df.rename(columns={"confusion": "confusion_matrix"}, inplace=True)
    elif "confusion_matrix" not in df.columns:
        df["confusion_matrix"] = "[]"

# === Rename precision/recall for consistency ===
for df in [xgb_df, cnn_df, resnet_df]:
    df.rename(columns={
        "precision_overall": "precision",
        "recall_overall": "recall"
    }, inplace=True)

# === Combine all ===
combined = pd.concat([rf_pixel, rf_pca, rf_hog, xgb_df, cnn_df, resnet_df], ignore_index=True)
combined.fillna(0.0, inplace=True)
combined.to_csv(os.path.join(metrics_dir, "combined_report_metrics.csv"), index=False)

print("Saved combined_app_metrics.csv with shape:", combined.shape)

KeyError: 'HOG'